In [1]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

C:\Users\Robotmedia9\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [2]:

tf.set_random_seed(777)  # for reproducibility
learning_rate = 0.1

nb_classes = 10
neural_net_size=100
# MNIST data image of shape 28 * 28 = 784
X = tf.placeholder(tf.float32, [None, 784])
# 0 - 9 digits recognition = 10 classes
Y = tf.placeholder(tf.float32, [None, nb_classes])

with tf.name_scope("layer1") as scope:
    W = tf.Variable(tf.random_normal([784, neural_net_size]))
    b = tf.Variable(tf.random_normal([neural_net_size]))
    layer1=tf.sigmoid(tf.matmul(X,W)+b)
    
    w1_hist = tf.summary.histogram("weights1", W)
    b1_hist = tf.summary.histogram("biases1", b)
    layer1_hist = tf.summary.histogram("layer1", layer1)
    
with tf.name_scope("layer2") as scope:

    W2 = tf.Variable(tf.random_normal([neural_net_size, neural_net_size]))
    b2 = tf.Variable(tf.random_normal([neural_net_size]))
    layer2=tf.sigmoid(tf.matmul(layer1,W2)+b2)
    
    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    layer2_hist = tf.summary.histogram("layer2", layer2)
    
with tf.name_scope("layer3") as scope:
    
    W3 = tf.Variable(tf.random_normal([neural_net_size, nb_classes]))
    b3 = tf.Variable(tf.random_normal([nb_classes]))
    layer3=tf.sigmoid(tf.matmul(layer2,W3)+b3)
    
    w3_hist = tf.summary.histogram("weights3", W3)
    b3_hist = tf.summary.histogram("biases3", b3)
    layer3_hist = tf.summary.histogram("layer3", layer3)
    
with tf.name_scope("hypothesis") as scope:    
    # Hypothesis (using softmax)
    hypothesis = tf.nn.softmax(layer3)
    layer3_hist = tf.summary.histogram("hypothesis", hypothesis)
    
# cost/loss function
with tf.name_scope("cost") as scope:
    cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
    cost_summ = tf.summary.scalar("cost", cost)
with tf.name_scope("train") as scope:
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Test model
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# parameters
training_epochs = 10
batch_size = 100

sess=tf.InteractiveSession()
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("./logs/deep_mnist_cost_r0_1")
writer.add_graph(sess.graph)  # Show the graph

# Initialize TensorFlow variables    
sess.run(tf.global_variables_initializer())


# Training cycle
step=0
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        summary,c, _ = sess.run([merged_summary,cost, optimizer], feed_dict={
                        X: batch_xs, Y: batch_ys})
        writer.add_summary(summary, global_step=step)
        step+=1
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1),
            'cost =', '{:.9f}'.format(avg_cost))

print("Learning finished")


Epoch: 0001 cost = 2.008425544
Epoch: 0002 cost = 1.997088674
Epoch: 0003 cost = 2.003479385
Epoch: 0004 cost = 2.007758895
Epoch: 0005 cost = 2.023300255


KeyboardInterrupt: 

In [ ]:

# Test the model using test sets
print("Accuracy: ", accuracy.eval(session=sess, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1]}))